In [43]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

In [44]:
def clean_names(names):
    return names.str.lower().str.replace(' ', '').str.replace('-','')

## Top 100 mineable coins

In [45]:
# Top 100 mineable coins
url = 'https://coinmarketcap.com/coins/views/filter-non-mineable/'

r = requests.get(url) 

page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
table = soup.find_all('div', class_='cmc-table__table-wrapper-outer')

In [48]:
# Get column names
column_names = []
for header in table[0].find_all('th'):
    column_names.append(str(header.string))
# Drop graphs
column_names.pop()
column_names.pop()

data = []
for row in table[2].find_all('tr')[1:]:
    new_row = []
    for entry in row:
        new_row.append(entry.text)
    new_row.pop()
    new_row.pop()
    data.append(new_row)
mineable_100 = pd.DataFrame(data, columns = column_names)
mineable_100 = mineable_100.set_index('Name')
mineable_100 = mineable_100.set_index(clean_names(mineable_100.index))
mineable_100 = mineable_100[['#', "Market Cap", "Price"]]
mineable_100 = mineable_100.rename(columns={'Market Cap': 'market_cap', 'Price':'price'})

## What to mine, nethash

### ASIC

In [51]:
url = 'https://whattomine.com/asic.json'

r = requests.get(url) 
data = r.json()

whattomine_asic = pd.DataFrame(data)
# normalize + clean names
whattomine_asic = pd.json_normalize(whattomine_asic['coins']).set_index(clean_names(whattomine_asic.index.str.lower()))
whattomine_asic['block_time'] = pd.to_numeric(whattomine_asic['block_time'])
whattomine_asic['nethash'] = pd.to_numeric(whattomine_asic['nethash'])
# Drop some columns
whattomine_asic = whattomine_asic[['tag','algorithm', 'block_time', 'difficulty', 'nethash', 'block_reward', 'exchange_rate', 'exchange_rate_curr']]
whattomine_asic[:4]

,tag,algorithm,block_time,difficulty,nethash,block_reward,exchange_rate,exchange_rate_curr
actinium,ACM,Lyra2z,154.0,3.960979e+03,1.104693e+11,50.0000,0.000002,BTC
axe,AXE,X11,158.0,1.208050e+06,3.283884e+13,1.9400,0.000045,BTC
bitcoin,BTC,SHA-256,765.0,1.655292e+13,9.293368e+19,13.0842,5282.020000,BTC
bitcoincashabc,BCH,SHA-256,588.0,5.378113e+11,3.928371e+18,12.5000,0.034176,BTC


### GPU

In [34]:
url = 'https://whattomine.com/coins.json'

r = requests.get(url) 
data = r.json()

whattomine_gpu = pd.DataFrame(data)
whattomine_gpu = pd.json_normalize(whattomine_gpu['coins']).set_index(clean_names(whattomine_gpu.index.str.lower()))
whattomine_gpu['block_time'] = pd.to_numeric(whattomine_gpu['block_time'])
whattomine_gpu['nethash'] = pd.to_numeric(whattomine_gpu['nethash'])
# Drop some columns
whattomine_gpu = whattomine_gpu[['tag','algorithm', 'block_time', 'difficulty', 'nethash', 'block_reward', 'exchange_rate', 'exchange_rate_curr']]
whattomine_gpu[:4]

,tag,algorithm,block_time,difficulty,nethash,block_reward,exchange_rate,exchange_rate_curr
aeternity,AE,CuckooCycle,185.0,2.912460e+07,157430,186.21900,1.694000e-05,BTC
aion,AION,"Equihash (210,9)",11.0,1.382876e+07,1257160,4.50000,9.920000e-06,BTC
anon,ANON,EquihashZero,150.0,3.396062e+05,2264,6.53125,9.400000e-07,BTC
beam,BEAM,BeamHashII,60.0,1.165186e+08,1941976,40.00000,4.600000e-05,BTC


## CoinWarz

In [35]:
url = 'https://www.coinwarz.com/v1/api/profitability?apikey=58467a6f2e2d4c65b7ef720b26137453&algo=all'
r = requests.get(url) 
data = r.json()

coinwarz = pd.DataFrame(data)
    
coinwarz = pd.json_normalize(coinwarz['Data'])
coinwarz = coinwarz.set_index(clean_names(coinwarz['CoinName']))

# Drop and rename some columns
coinwarz = coinwarz[['CoinTag', 'Algorithm','BlockTimeInSeconds', 'Difficulty', 'BlockReward', 'ExchangeRate']]
coinwarz = coinwarz.rename(columns={'CoinTag': 'tag', 'Algorithm': 'algorithm','BlockTimeInSeconds':'block_time',
                                    'Difficulty': 'difficulty', 'BlockReward': 'block_reward', 'ExchangeRate': 'exchange_rate'})
coinwarz['nethash_scraped'] = 0
coinwarz[:4]

,tag,algorithm,block_time,difficulty,block_reward,exchange_rate,nethash_scraped
CoinName,,,,,,,
ethereumclassic,ETC,EtHash,15,1.432911e+14,4.0,8.739000e-04,0
ethereum,ETH,EtHash,15,2.085399e+15,2.0,2.209034e-02,0
cannabiscoin,CANN,X11,42,2.351116e+04,70.0,3.100000e-07,0
verge,XVG,Scrypt,30,1.118767e+04,730.0,3.800000e-07,0


In [36]:
coinwarz.loc['bitcoin']

tag                        BTC
algorithm              SHA-256
block_time                 600
difficulty         1.65529e+13
block_reward              12.5
exchange_rate          5299.69
nethash_scraped              0
Name: bitcoin, dtype: object

Here, the exchange rate is relative the bitcoin, except for bitcoin itself, where it is in US$

# Combining the data

In [37]:
def check_missing(data):
    missing_data = result[data.isnull().any(axis=1)]
    return missing_data.shape[0]

In [38]:
whattomine = whattomine_gpu.append(whattomine_asic)
whattomine = whattomine.rename(columns={'nethash':'nethash_scraped'})
whattomine

,tag,algorithm,block_time,difficulty,nethash_scraped,block_reward,exchange_rate,exchange_rate_curr
aeternity,AE,CuckooCycle,185.0,2.912460e+07,1.574300e+05,186.21900,1.694000e-05,BTC
aion,AION,"Equihash (210,9)",11.0,1.382876e+07,1.257160e+06,4.50000,9.920000e-06,BTC
anon,ANON,EquihashZero,150.0,3.396062e+05,2.264000e+03,6.53125,9.400000e-07,BTC
beam,BEAM,BeamHashII,60.0,1.165186e+08,1.941976e+06,40.00000,4.600000e-05,BTC
bitcash,BITC,X25X,63.0,3.200000e+01,2.181571e+09,19.35000,1.120000e-06,BTC
...,...,...,...,...,...,...,...,...
vergegroestl,XVG,Myr-Groestl,150.0,7.377774e+06,2.112486e+14,400.00000,3.800000e-07,BTC
vergelyra2rev2,XVG,Lyra2REv2,150.0,5.344045e+04,1.530166e+12,400.00000,3.800000e-07,BTC
vergescrypt,XVG,Scrypt,150.0,1.118767e+04,3.203378e+11,400.00000,3.800000e-07,BTC
viacoin,VIA,Scrypt,24.0,2.896865e+05,5.184143e+13,0.62500,1.681000e-05,BTC


In [39]:
result = mineable_100.join(whattomine, how="left")

In [40]:
for coin in coinwarz.index:
    if coin not in whattomine.index.values:
        if coin in result.index:
            result.loc[coin, ['tag','algorithm','block_time','difficulty','nethash_scraped']] = coinwarz.loc[coin]
check_missing(result)

67

In [41]:
result[:10]

,#,market_cap,price,tag,algorithm,block_time,difficulty,nethash_scraped,block_reward,exchange_rate,exchange_rate_curr
Name,,,,,,,,,,,
bitcoin,1,"$96,857,634,339","$5,300.20",BTC,SHA-256,787.0000,1.655292e+13,9.033579e+19,13.087200,5246.430000,BTC
ethereum,2,"$12,862,918,727",$116.80,ETH,Ethash,13.3475,2.084378e+15,1.561625e+14,2.000000,0.022069,BTC
bitcoincash,3,"$3,334,576,134",$181.87,BCH,SHA-256,600.0000,5.455478e+11,0.000000e+00,NaN,NaN,NaN
litecoin,4,"$2,232,848,115",$34.72,LTC,Scrypt,156.0000,5.152158e+06,1.418484e+14,12.500000,0.006562,BTC
bitcoinsv,5,"$2,181,597,495",$119.00,BSV,SHA-256,583.0000,3.619383e+11,2.666404e+18,12.500000,0.022276,BTC
tron,6,"$673,404,720",$0.010099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cardano,7,"$670,144,840",$0.025847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monero,8,"$635,486,108",$36.34,XMR,RandomX,120.0000,1.395270e+11,1.162725e+09,1.868363,0.006851,BTC
ethereumclassic,9,"$536,203,208",$4.61,ETC,Ethash,13.5743,1.430807e+14,1.054056e+13,3.104000,0.000874,BTC


In [42]:
result.to_csv('data/mineable_100.csv')